### Classification 데이터변경

In [1]:
# customerdata

### 라이브러리 선언 및 데이터 불러오기

In [2]:
# 숫자 및 데이터 다루는 라이브러리
import numpy as np
import pandas as pd

# 분류모델 선언
from sklearn import svm

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# 훈련/테스트 데이터 자동 분리
from sklearn.model_selection import train_test_split

In [5]:
# 분류모델 정확도 라이브러리 선언
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

### 데이터 불러오기

In [6]:
dataUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/pima-indians-diabetes.csv"

In [7]:
inData = pd.read_csv(dataUrl)

In [8]:
inData.head(2)

,pregnancies,glucose,diastolic,trceps,insulin,bmi,dpf,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [9]:
# # csv 파일 불러오기
# csData = pd.read_csv("../dataset/customer.csv")
# # csv 파일 불러오기
# csData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/customer.csv")

### 1. 타입 통합 / 숫자형 컬럼 추가

In [10]:
inData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pregnancies  768 non-null    int64  
 1   glucose      768 non-null    int64  
 2   diastolic    768 non-null    int64  
 3   trceps       768 non-null    int64  
 4   insulin      768 non-null    int64  
 5   bmi          768 non-null    float64
 6   dpf          768 non-null    float64
 7   age          768 non-null    int64  
 8   diabetes     768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [11]:
inDataColumns = inData.columns

In [12]:
intColumns = ["pregnancies","glucose","diastolic","trceps","insulin","age","diabetes"]

In [13]:
floatColumns = ["bmi","dpf"]

In [14]:
for i in range(0, len(intColumns)):
    inData[intColumns[i]] = inData[intColumns[i]].astype(int)

In [15]:
for i in range(0, len(floatColumns)):
    inData[floatColumns[i]] = inData[floatColumns[i]].astype(float)

In [16]:
inData.dtypes

pregnancies      int32
glucose          int32
diastolic        int32
trceps           int32
insulin          int32
bmi            float64
dpf            float64
age              int32
diabetes         int32
dtype: object

### 2.  특성 선정 / 데이터 분리

In [17]:
stdCorr = 0.2

In [18]:
corrDf = inData.corr()

In [19]:
corrDf

,pregnancies,glucose,diastolic,trceps,insulin,bmi,dpf,age,diabetes
pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
diastolic,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
trceps,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
bmi,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
dpf,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
diabetes,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [20]:
label = ["diabetes"]

In [21]:
features = \
    list( corrDf.loc[ (abs( corrDf["diabetes"] ) > stdCorr ) &\
                     (corrDf["diabetes"] != 1 ) ].index )

In [22]:
features

['pregnancies', 'glucose', 'bmi', 'age']

In [24]:
trainingData_features,\
testData_features,\
trainingData_label,\
testData_label =\
train_test_split(inData.loc[:, features], 
                 inData.loc[:, label], 
                 test_size=0.2, 
                 random_state=1)

In [25]:
print(trainingData_features.shape)
print(testData_features.shape)
print(trainingData_label.shape)
print(testData_label.shape)

(614, 4)
(154, 4)
(614, 1)
(154, 1)


### 3. 모델 선언/학습

In [26]:
model_method = KNeighborsClassifier(n_neighbors=3)

In [27]:
model_knn = model_method.fit(trainingData_features, trainingData_label)

C:\Users\kopo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


### 4. 예측

In [28]:
predictValue = model_knn.predict(testData_features)

### 5. 데이터 정리

In [29]:
testDataAll = inData.loc[testData_label.index]

In [30]:
testDataAll.shape

(154, 9)

In [31]:
len(predictValue)

154

In [32]:
testDataAll["PREDICT_KNN"] = predictValue

In [33]:
accuracy_score(y_true=testDataAll.diabetes,
               y_pred=testDataAll.PREDICT_KNN)

0.7077922077922078